---
## Module import

앞으로 사용 할 기본적인 모듈들을 불러옵니다.  
(사실 일종의 세트처럼 준비한 소스들이라서, 지금 프로젝트에서 쓰지 않더라도 import한 것들이 많습니다.)

In [1]:
# 앞으로 사용 할 기본적인 모듈들 Import

import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import re 


import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)
%matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Google Drive - Colab  연동을 위한 라이브러리 Import
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Exploration/data'

# PIL 라이브러리 Import
from PIL import Image

import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다

Mounted at /content/drive


---
## Data load

미리 준비된 50여개의 가사 txt 파일들을 불러옵니다.

In [2]:
import glob
import os

txt_file_path = data_dir+'/lyrics/*'

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


불러온 가사들을 행단위로 확인해보려합니다.

In [3]:
print(raw_corpus[:9])

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes"]


In [4]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


전처리로써, 특수문자를 제거하고 문장을 정제하는 과정이 필요합니다.  
데이터의 결측치를 제거하는 것과도 같은 과정이라고 생각하면 됩니다.

In [5]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


corpus 배열에 정제된 문장들을 입력합니다.

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

결과를 보니 start - 가사 - end의 규칙으로 가사들이 잘 정리되었습니다.

컴퓨터의 입장에서 '단어'는 단순한 텍스트이기 때문에 연산이 가능한 수로써 변환해주는 작업이 필요합니다.  
이러한 과정을 tokenizing이라 하는데, 이는 number와 word가 쌍으로 매칭되게끔 처리해주는 과정입니다.

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2 673  27 ...   0   0   0]
 [  2 673  27 ...   0   0   0]
 [  2 673  27 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd73e6d95d0>


In [8]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


1~3까지는 unk, start, end가 기본적으로 토큰화되었고 그 다음으로 각각의 단어들이 이루어짐을 확인 할 수 있습니다.

그렇다면 이제 소스src와 타겟tgt을 한번 비교해보겠습니다.

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

소스는 start를 의미하는 2로 시작하지만 타겟은 2로 시작하지 않고 왼쪽으로 1만큼 은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 shift 되어있는 형태임을 확인 할 수 있습니다.

이따가 사용할 dataset을 구성해주도록 하겠습니다.

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 346), dtype=tf.int32, name=None), TensorSpec(shape=(256, 346), dtype=tf.int32, name=None))>

여기까지 기본적인 전처리와 사전작업은 끝났습니다.

---
## Train set / Validation set 분리

In [11]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

---
## 모델 디자인 및 학습

tf.keras.model을 subclassing하는 방식으로 모델을 준비하겠습니다.  
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있습니다.

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 19 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 2048 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

모델의 빌드 전 sample dataset으로 모델을 확인하는 과정입니다.

In [13]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 346, 7001), dtype=float32, numpy=
array([[[-6.42577725e-05, -9.85150546e-05,  8.15259136e-06, ...,
         -5.58032625e-05,  7.05472194e-05, -6.63073588e-05],
        [ 2.20253278e-05, -7.47789381e-05, -2.09692662e-05, ...,
         -6.00451995e-06,  1.65784528e-04, -3.71823298e-05],
        [ 1.60003096e-04, -1.15399336e-04, -1.02662685e-04, ...,
          4.19186945e-05,  2.55620951e-04,  2.98975847e-05],
        ...,
        [ 9.51000955e-04,  3.09340830e-04, -3.71783281e-05, ...,
          1.68812135e-03,  1.31960819e-03, -4.92806248e-05],
        [ 9.50954040e-04,  3.09395138e-04, -3.72436662e-05, ...,
          1.68812764e-03,  1.31956849e-03, -4.93675652e-05],
        [ 9.50882444e-04,  3.09341995e-04, -3.72567847e-05, ...,
          1.68808841e-03,  1.31968840e-03, -4.93758162e-05]],

       [[-6.42577725e-05, -9.85150546e-05,  8.15259136e-06, ...,
         -5.58032625e-05,  7.05472194e-05, -6.63073588e-05],
        [-1.24948390e-04, -1.23893667e-04, -1

In [14]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  133019    
                                                                 
 lstm (LSTM)                 multiple                  16941056  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  14345049  
                                                                 
Total params: 64,981,748
Trainable params: 64,981,748
Non-trainable params: 0
_________________________________________________________________


그런데 호출해 보니 그동안 많이 보았던 것과는 다른 점이 있습니다. 우리가 궁금했던 Output Shape를 정확하게 알려주지 않습니다. 바로 위에서 설명한 이유 때문입니다. 우리의 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없는 것입니다. -LMS said

이제 모델을 학습시킵니다.

In [15]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 346), dtype=tf.int32, name=None), TensorSpec(shape=(256, 346), dtype=tf.int32, name=None))>

In [19]:
# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy']) # 손실함수와 훈련과정을 설정했다.
# model.fit(dataset, batch_size=16, epochs=1) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

model.fit(enc_train, 
          dec_train, 
          batch_size=16,
          validation_data=val_dataset,
          epochs=5,
          verbose=1
          )

tf.test.is_gpu_available()

Epoch 1/5
8788/8788 [==============================] - 1262s 143ms/step - loss: 0.1373 - accuracy: 0.9775 - val_loss: 0.1274 - val_accuracy: 0.9782
Epoch 2/5
8788/8788 [==============================] - 1260s 143ms/step - loss: 0.1206 - accuracy: 0.9789 - val_loss: 0.1189 - val_accuracy: 0.9792
Epoch 3/5
8788/8788 [==============================] - 1262s 144ms/step - loss: 0.1089 - accuracy: 0.9800 - val_loss: 0.1132 - val_accuracy: 0.9801
Epoch 4/5
8788/8788 [==============================] - 1260s 143ms/step - loss: 0.0975 - accuracy: 0.9814 - val_loss: 0.1097 - val_accuracy: 0.9809
Epoch 5/5
8788/8788 [==============================] - 1260s 143ms/step - loss: 0.0869 - accuracy: 0.9829 - val_loss: 0.1078 - val_accuracy: 0.9816


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

epoch를 거듭하며 loss가 내려감을 확인 할 수 있다.

이제 모델이 문장을 생성하게 해보자.

In [20]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [27]:
generate_text(model, tokenizer, init_sentence="<start> i miss", max_len=30)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 i miss 로 시작되는 문장을 생성

'<start> i miss you , miss you <end> '

input : i miss  
output : i miss you , miss you  
납득 가능하지만 다소 so far

In [29]:
generate_text(model, tokenizer, init_sentence="<start> i still", max_len=30)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 i still 로 시작되는 문장을 생성

'<start> i still don t know what to tell you now <end> '

input : i still  
output : i still don t know what to tell you now  
예상 외로, 간단한 출력만으로 멋진 출력을 얻었다.

---
## Discussion

1. 어떻게 보면 가장 '인공지능'이라고 느낄 수 있었던 프로젝트였다. 내가 입력하지 않은 내용들(물론 소스가 되는 가사가 있었지만, 이것은 내가 사전에 미리 확인하지 않은 massive data였을 뿐이다)을 이용해 내가 예상하지 못한 결과물(모델이 학습 후 결정한 결과)을 얻는 것은 매우 신기한 경험이었고 고무적이었다.
2. 모든 내용을 이해한 것은 아니지만 LMS에서 제공하는 소스코드만으로 충분히 프로젝트를 진행할 수 있어서, 코드에 대한 스트레스 없이 전반적인 학습과정을 이해하는데 집중 할 수 있어서 더 좋았다.
3. 다만 좀더 다양한 시도를 해보고 싶었지만 개인적인 사정으로 마감기한에 가까워져서야 시작하는 바람에 (시간적인)리소스 문제로 이것저것 더 해보지 못해서 아쉬웠다. 매번 느끼고 매번 하는 다짐이지만 1.시간 관리 좀 잘 하고 2.좀더 열심히 집중해서 3.학습에 더 많은 시간을 쓰는 사람이 되고싶다 제발. ㅠㅠ